In [ ]:
import gradio as gr
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
import os

# === Charger modèle et données en toute sécurité ===
try:
    model = load_model('D:/RATRAPAGE/heart_model.h5')
except Exception as e:
    raise FileNotFoundError(f"Erreur lors du chargement du modèle : {e}")

try:
    df = pd.read_excel('D:/cours IFI/Fouille_Donees/dataset/archive1/heart_excel.xlsx')
except Exception as e:
    raise FileNotFoundError(f"Erreur lors du chargement des données : {e}")

# === Encodage des variables catégorielles ===
df_encoded = pd.get_dummies(df.drop("HeartDisease", axis=1), drop_first=True)

# === Initialisation du scaler ===
scaler = StandardScaler()
scaler.fit(df_encoded)

# === Fonctions d'analyse exploratoire ===
def eda_plot():
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm", fmt=".2f", ax=ax)
    plt.title("Matrice de corrélation")
    return fig

def summary():
    return df.describe().T

# === Fonction de prédiction ===
def predict_disease(age, sex, resting_bp, cholesterol, fasting_bs, max_hr, oldpeak,
                    cpt_ata, cpt_nap, cpt_ta, ecg_normal, ecg_st,
                    exercise_angina_y, st_flat, st_up):
    
    input_data = pd.DataFrame([{
        "Age": age,
        "Sex": sex,
        "RestingBP": resting_bp,
        "Cholesterol": cholesterol,
        "FastingBS": fasting_bs,
        "MaxHR": max_hr,
        "Oldpeak": oldpeak,
        "ChestPainType_ATA": int(cpt_ata),
        "ChestPainType_NAP": int(cpt_nap),
        "ChestPainType_TA": int(cpt_ta),
        "RestingECG_Normal": int(ecg_normal),
        "RestingECG_ST": int(ecg_st),
        "ExerciseAngina_Y": int(exercise_angina_y),
        "ST_Slope_Flat": int(st_flat),
        "ST_Slope_Up": int(st_up)
    }])

    try:
        scaled_input = scaler.transform(input_data)
        prediction = model.predict(scaled_input)[0][0]
        pred_label = "🩺 Risque de maladie cardiaque" if prediction > 0.5 else "✅ Pas de risque détecté"
        return f"{pred_label} (Probabilité : {prediction:.2f})"
    except Exception as e:
        return f"Erreur de prédiction : {str(e)}"

# === Interface Gradio ===

# Onglet EDA
with gr.Blocks() as eda_tab:
    gr.Markdown("## 🔍 Analyse Exploratoire")
    with gr.Row():
        summary_output = gr.Dataframe()
        correlation_output = gr.Plot()

    btn_summary = gr.Button("📊 Afficher Résumé")
    btn_corr = gr.Button("🔗 Afficher Matrice de corrélation")

    btn_summary.click(fn=summary, outputs=summary_output)
    btn_corr.click(fn=eda_plot, outputs=correlation_output)

# Onglet Prédiction
with gr.Blocks() as predict_tab:
    gr.Markdown("## 🧠 Prédiction de Maladie Cardiaque")
    with gr.Row():
        age = gr.Number(label="Âge")
        sex = gr.Radio(choices=[0, 1], label="Sexe (0=F, 1=M)")

    with gr.Row():
        resting_bp = gr.Number(label="Pression Artérielle au Repos")
        cholesterol = gr.Number(label="Cholestérol")

    with gr.Row():
        fasting_bs = gr.Radio(choices=[0, 1], label="Glycémie à jeun (>120 mg/dl)")
        max_hr = gr.Number(label="Fréquence Cardiaque Max")

    oldpeak = gr.Number(label="Oldpeak (dépression ST)")

    gr.Markdown("### 🫀 Type de Douleur Thoracique")
    cpt_ata = gr.Checkbox(label="Angine typique (ATA)")
    cpt_nap = gr.Checkbox(label="Douleur non angineuse (NAP)")
    cpt_ta = gr.Checkbox(label="Angine atypique (TA)")

    gr.Markdown("### 📉 Résultat ECG")
    ecg_normal = gr.Checkbox(label="Normal")
    ecg_st = gr.Checkbox(label="Anomalie ST")

    gr.Markdown("### 🏃‍♂️ Angine à l'Effort")
    exercise_angina_y = gr.Checkbox(label="Angine induite par exercice")

    gr.Markdown("### 📈 Pente du Segment ST")
    st_flat = gr.Checkbox(label="Plate")
    st_up = gr.Checkbox(label="Montante")

    prediction_output = gr.Textbox(label="Résultat")

    btn_predict = gr.Button("🔮 Prédire")
    btn_predict.click(
        fn=predict_disease,
        inputs=[age, sex, resting_bp, cholesterol, fasting_bs, max_hr, oldpeak,
                cpt_ata, cpt_nap, cpt_ta, ecg_normal, ecg_st,
                exercise_angina_y, st_flat, st_up],
        outputs=prediction_output
    )

# Application multi-onglet avec configuration robuste
demo = gr.TabbedInterface(
    interface_list=[eda_tab, predict_tab],
    title="🧬 Application d'Analyse Cardiaque"
)

# === Lancement protégé ===
demo.launch(
    server_port=7861,  # changer si nécessaire
    inbrowser=True,
    debug=True          # pour voir les erreurs détaillées
)


Running on local URL:  http://127.0.0.1:7861
IMPORTANT: You are using gradio version 4.25.0, however version 4.44.1 is available, please upgrade.
--------

To create a public link, set `share=True` in `launch()`.
IMPORTANT: You are using gradio version 4.25.0, however version 4.44.1 is available, please upgrade.
--------
IMPORTANT: You are using gradio version 4.25.0, however version 4.44.1 is available, please upgrade.
--------


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\hermanhk\anaconda3\Lib\site-packages\pydantic\type_adapter.py", line 277, in _init_core_attrs
    self._core_schema = _getattr_no_parents(self._type, '__pydantic_core_schema__')
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hermanhk\anaconda3\Lib\site-packages\pydantic\type_adapter.py", line 119, in _getattr_no_parents
    raise AttributeError(attribute)
AttributeError: __pydantic_core_schema__

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\hermanhk\anaconda3\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 409, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hermanhk\anaconda3\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call_